In [1]:
!pip install afinn 

In [2]:
!pip install textblob

In [3]:
!pip install textblob_fr

In [4]:
!pip install plotly

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from afinn import Afinn 
import spacy
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import re
from spacy.lang.fr.stop_words import STOP_WORDS
import string
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
import plotly.graph_objects as go
import plotly.express as px

In [6]:
#df = pd.read_csv("C://Users//ganci//Documents//master_miashs sans TER//Analyse de données textuelles//Projet//Analyse_d_opinion//Données//corpus_bogdanof.csv")
df = pd.read_csv("C://Users//YCarl//OneDrive//Bureau//Analyse_d_opinion//Données//corpus_bogdanof.csv")

In [7]:
df

,Unnamed: 0,post,sujet
0,0,"\r\nPour corriger les passages tendancieux, je...",frerebogdanof_travaux
1,1,\r\nAssez d'accord.\r\nJ'ai fait quelques refo...,frerebogdanof_travaux
2,2,\r\nIl semble bien que le drame soit plus auto...,frerebogdanof_travaux
3,3,\r\nJe suis aussi d'accord pour dire qu'il y a...,frerebogdanof_travaux
4,4,\r\nJe vois pas trop le dÃ©bat avancer. Ne ser...,frerebogdanof_travaux
...,...,...,...
271,271,"\r\n\r\nEn effet , pas très encyclopédique et ...",frerebogdanof_travaux
272,272,"\r\n\r\nJe trouve l'article assez faible, surt...",frerebogdanof_travaux
273,273,"\r\n\r\nPour information : Sylvestre Huet, «\r...",frerebogdanof_travaux
274,274,"\r\n\r\n\r\n\r\nSalut tout le monde,\r\n\r\nDa...",frerebogdanof_travaux


In [8]:
df = df.replace(to_replace='\n', value='', regex=True)

In [9]:
df = df.replace(to_replace='Ã©', value='é', regex=True)

In [10]:
df= df.drop(['Unnamed: 0'],axis=1)
df

,post,sujet
0,"\rPour corriger les passages tendancieux, je p...",frerebogdanof_travaux
1,\rAssez d'accord.\rJ'ai fait quelques reformul...,frerebogdanof_travaux
2,\rIl semble bien que le drame soit plus autour...,frerebogdanof_travaux
3,\rJe suis aussi d'accord pour dire qu'il y atr...,frerebogdanof_travaux
4,\rJe vois pas trop le débat avancer. Ne serait...,frerebogdanof_travaux
...,...,...
271,"\r\rEn effet , pas très encyclopédique et pas ...",frerebogdanof_travaux
272,"\r\rJe trouve l'article assez faible, surtout ...",frerebogdanof_travaux
273,"\r\rPour information : Sylvestre Huet, «\rUn c...",frerebogdanof_travaux
274,"\r\r\r\rSalut tout le monde,\r\rDans l'interve...",frerebogdanof_travaux


In [11]:
NAN = [(c, df[c].isna().mean()*100) for c in df]
NAN = pd.DataFrame(NAN, columns=["column_name", "percentage"])
NAN.sort_values("percentage", ascending=False)

,column_name,percentage
0,post,0.0
1,sujet,0.0


In [12]:
df["post"]= df["post"].str.lower()

## tokenisation des poste

In [13]:
tokenpost=[]
for comment in df["post"].apply(str):
    Word_Tok = []
    for word in  re.sub("\W"," ",comment ).split():
        Word_Tok.append(word)
    tokenpost.append(Word_Tok)
    

In [14]:
tokenpost=[]
for comment in df["post"].apply(str):
    Word_Tok = []
    for word in  re.sub("\W"," ",comment ).split():
        Word_Tok.append(word)
    tokenpost.append(Word_Tok)
    

In [15]:
df["tokenpost"]= tokenpost
df.head()

,post,sujet,tokenpost
0,"\rpour corriger les passages tendancieux, je p...",frerebogdanof_travaux,"[pour, corriger, les, passages, tendancieux, j..."
1,\rassez d'accord.\rj'ai fait quelques reformul...,frerebogdanof_travaux,"[assez, d, accord, j, ai, fait, quelques, refo..."
2,\ril semble bien que le drame soit plus autour...,frerebogdanof_travaux,"[il, semble, bien, que, le, drame, soit, plus,..."
3,\rje suis aussi d'accord pour dire qu'il y atr...,frerebogdanof_travaux,"[je, suis, aussi, d, accord, pour, dire, qu, i..."
4,\rje vois pas trop le débat avancer. ne serait...,frerebogdanof_travaux,"[je, vois, pas, trop, le, débat, avancer, ne, ..."


## Suppression des negations avec le stopwords de spacy

In [16]:
stop_words=set(STOP_WORDS)

## supression des mots de negations de mots vide afin(important pour la suite de l'analyse de sentement)
deselect_stop_words = ['n\'', 'ne','pas','plus','personne','aucun','ni','aucune','rien']
for w in deselect_stop_words:
    if w in stop_words:
        stop_words.remove(w)
    else:
        continue

### suppression de mots vides de sens 

In [17]:
postfiltre=[]
for comment in df["tokenpost"]:
    filteredComment = [w for w in comment if not ((w in stop_words) or (len(w) == 1))]
    postfiltre.append(' '.join(filteredComment))

In [18]:
df["postfinal"]=postfiltre
df.postfinal[56]

'dites oui doute compétent juger erreur mathématique ici pas cas prononcer répète renvoyer dos dos intervenants polémique pas faire acte neutralité directeurs thèse frères éminents savants ne vois pas trancher répète ne agit pas querelle experts ne saura jamais fin mot histoire sachant qu sciences dures pas pas vérité définitive erreur prouvée non fois non science distingue connaissances justement démarche consiste parfaitement définir objets définit cadre cadre vérité mieux avez justement remarquer niveau débat jamais lieu précisément niveau thèse jugée digne soutenue texte mathématique pires attaques plan scientifique ne sais pas compétent juger ne pas traquent bogdanoff forums soient mieux placés faire non plus êtes pas neutre bases appuyez propos véracité jugée aune véhément mettre doute valeur argument revient traquent jugement partial avez participé débats faut rester faits effectivement soutenu thèses succès éclat plein pas éléments pretent flan critiques effectivement tollé con

# Analyse de sentiment par documents

In [19]:
senti_list = []
for i in df["postfinal"]:
    vs = tb(i).sentiment[0]
    if (vs > 0):
        senti_list.append('Positive')
    elif (vs < 0):
        senti_list.append('Negative')
    else:
        senti_list.append('Neutral')   

In [20]:
df["sentiment"]=senti_list
df.head()

,post,sujet,tokenpost,postfinal,sentiment
0,"\rpour corriger les passages tendancieux, je p...",frerebogdanof_travaux,"[pour, corriger, les, passages, tendancieux, j...",corriger passages tendancieux propose modifica...,Positive
1,\rassez d'accord.\rj'ai fait quelques reformul...,frerebogdanof_travaux,"[assez, d, accord, j, ai, fait, quelques, refo...",accord reformulations sens pas fin compte arti...,Negative
2,\ril semble bien que le drame soit plus autour...,frerebogdanof_travaux,"[il, semble, bien, que, le, drame, soit, plus,...",bien drame plus autour article mãªme lignes ré...,Positive
3,\rje suis aussi d'accord pour dire qu'il y atr...,frerebogdanof_travaux,"[je, suis, aussi, d, accord, pour, dire, qu, i...",accord qu atrop anti bogdanov page non signé 7...,Negative
4,\rje vois pas trop le débat avancer. ne serait...,frerebogdanof_travaux,"[je, vois, pas, trop, le, débat, avancer, ne, ...",vois pas trop débat avancer ne pas temps clore...,Positive


In [23]:
df.to_csv('C://Users//YCarl//OneDrive//Bureau//Analyse_d_opinion//Données//data_fichierSpacy.csv',
           encoding = 'utf-8',
           header = True)

## Data Visualization

In [ ]:
Number_sentiment= df.groupby(["sentiment"]).count().reset_index().reset_index(drop=True)

In [ ]:
fig = px.histogram(df, x="sentiment",color="sentiment")
fig.update_layout(
    title_text='Sentiment of reviews', # title of plot
    xaxis_title_text='Sentiment', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.2, 
    bargroupgap=0.1
)
fig.show()

# debut de l'analyse de sentiment avec AFINN

In [ ]:
from afinn import Afinn 
import pandas as pd 
  
afn = Afinn() 
  
news_df = ['les gens pensent aux chiens','je deteste les fleurs', 
         'il est gentils et intelligents'] 
           
scores = [afn.score(article) for article in news_df] 
sentiment = ['positive' if score > 0 
                          else 'negative' if score < 0 
                              else 'neutral' 
                                  for score in scores] 
      
df = pd.DataFrame() 
df['topic'] =  news_df 
df['scores'] = scores 
df['sentiments'] = sentiment 
print(df)

## text de polarité avec textblob


In [ ]:
!pip install textblob_fr

In [ ]:
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
import string
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

In [ ]:
news_df = ['les gens pensent aux chiens',"il n'aime pas les fleurs", 
         'il est gentils et intelligents'] 

In [ ]:
for i in news_df:
    print(tb(i).sentiment[0])
